In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

CPU-Pytorch (pretrained=False) TASK-2

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import timm  # Import the timm library for ViT
from sklearn.model_selection import train_test_split

# Define image preprocessing and data augmentation
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = 'task02'
all_data = ImageFolder(data_dir, data_transforms)

# Split the dataset into training and validation
train_data, val_data = train_test_split(all_data, test_size=0.2, random_state=42)

dataloaders = {
    'train': DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)
}

# Load the pre-trained ViT model using timm
model = timm.create_model('vit_base_patch16_224', pretrained=False)

# Modify the model for your classification task (e.g., change the output layer)
# You should replace this with the number of classes in your dataset
num_classes = len(all_data.classes)
model.head = nn.Linear(model.head.in_features, num_classes)
print(num_classes)
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Train the model
num_epochs = 10  # You can adjust the number of epochs
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        corrects = 0

        for inputs, labels in dataloaders[phase]:
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(all_data)
        epoch_acc = corrects.double() / len(all_data)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')



In [ ]:
# Inference with a single image
import cv2
from PIL import Image
from torchvision.transforms import ToTensor

# Load and preprocess the input image
image_path = 'TCGA-AA-3952-01Z-00-DX1.6a51a689-74cb-4204-9f7a-f5e3fc55fb2d_(11705,25708).jpg'
image = Image.open(image_path).convert('RGB')
image = data_transforms(image)
image = image.unsqueeze(0)  # Add batch dimension

# Make a prediction
model.eval()
with torch.no_grad():
    outputs = model(image)

# Get the class with the highest probability
_, predicted_class = torch.max(outputs, 1)
print(predicted_class.item())
